### ANN Classification

1)	Data Preprocess  
a)	Import the libraries, please use TensorFlow 2 (default version on Google Colab is 2.14.0) to build your model.  
b)	Load the dataset, use columns [3:-1] as the X set.
c)	Encode the categorical variables.  
d)	Train test split and feature scaling.  
 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_excel('../Data/Churn_Modelling.xlsx')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
from sklearn.preprocessing import LabelEncoder
x = data.iloc[:, 3:-1].values
y = data.iloc[:,-1].values

le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

2)	Build the Model  
a)	Build a keras sequential model with 2 more Dense Layers  
b)	Set the unit number and activation function of those layers  
c)	Add a Dense Layer as output layer, with unit = 1 and activation function = sigmoid.  


In [6]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

3)	Training  
a)	Compile the model with adam optimizer, binary_crossentropy as loss function, metrics = [‘accuracy’].  
b)	Fit the model on the train set with more than 100 epochs. (Optional: set an early stop)


In [7]:
ann.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
ann.fit(X_train,y_train, batch_size=32, epochs=105)

Epoch 1/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 317us/step - accuracy: 0.7176 - loss: 0.6178
Epoch 2/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.8034 - loss: 0.4817
Epoch 3/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step - accuracy: 0.8000 - loss: 0.4623
Epoch 4/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7963 - loss: 0.4532
Epoch 5/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step - accuracy: 0.7956 - loss: 0.4409
Epoch 6/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step - accuracy: 0.8026 - loss: 0.4341
Epoch 7/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 279us/step - accuracy: 0.8041 - loss: 0.4282
Epoch 8/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 259us/step - accuracy: 0.8087 - loss: 0.4207
Epoch 9/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 266us/step - accuracy: 0.8144 - loss: 0.4050
Epoch 10/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step - accuracy: 0.8132 - loss: 0.4007
Epoch 11/105
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step - accuracy: 0.8373 - loss: 0.3810
Epoch 12/105
250/25

4)	Predicting  
a)	Use our ANN model to predict if the customer with the following informations will leave the bank:   
b)	Predict the test set result, set the threshold as 0.5, and calculate the confusion matrix and accuracy score.  


In [9]:
# France = [1, 0, 0]Male = 0
customer_data = np.array([[1, 0, 0, 600, 0, 40, 3, 60000, 2, 1, 1, 50000]])
prediction = ann.predict(customer_data)
testing = (prediction > 0.5).astype(int)

if testing[0][0] == 1:
    print("Leave")
else:
    print("Stay.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Leave


In [10]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[1530   65]
 [ 198  207]]
0.8685


5)	Report  
a)	Write a brief summary of your model architecture, and the test result.


The model is an ANN built with Keras to predict whether a customer will leave the bank. There is:

- Input Layer: The input has been preprocessed to include 12 features after encoding categorical variables (e.g., Geography and Gender).
- 2 Hidden Layers: Helps capture non-linear relationships in the data.
- Output Layer: Computes a probability score, allowing the model to make the binary predictions that a customer leaves or stays
- Optimizer (Adam) ,Loss Function (Binary cross-entropy), and Metrics (Accuracy)
- The model was trained for 105 epochs with a batch size of 32.

Test Set Result  
- True Negatives: correctly predicted 1530 customers who stayed.  
- False Positives: incorrectly predicted 65  customers as leaving when they actually stayed.  
- False Negatives: incorrectly predicted 207 customers as staying when they actually left.  
- True Positives: correctly predicted 198 customers who left.  
- The accuracy score is 0.866, which is pretty high performing, 